In [1]:
!pip install finlab > log.txt

In [2]:
from finlab import data
from finlab.backtest import sim
from finlab.dataframe import FinlabDataFrame
import finlab
from finlab import login
from finlab import data
import numpy as np
import pandas as pd

In [3]:
import finlab
finlab.login('gwTx24QtR/eVkaIDWHg1cSsMNn/gosK0wPUSXHNhAcF4BAd7Ij7PQQc/fx/KSxD0#vip_m')

輸入成功!


In [4]:
def make_total_iterations(**params):
    return '_'.join(f'{k}_{v}' for k, v in params.items())

class Condition:

    def __init__(self):

        self.close =  data.get("price:收盤價")
        self.high =  data.get("price:最高價")
        self.low =  data.get("price:最低價")
        self.open =   data.get("price:開盤價")
        self.volume =  data.get("price:成交股數")
        # self.broker = data.get('broker_transactions')
        self.inv = data.get('inventory')
        self.close = data.get("price:收盤價")
        self.revenue = data.get('monthly_revenue:當月營收')
        self.rev_year_growth = data.get('monthly_revenue:去年同月增減(%)')
        self.rev_month_growth = data.get('monthly_revenue:上月比較增減(%)')
        self.pe = data.get('price_earning_ratio:本益比')
        self.營業利益成長率 = data.get('fundamental_features:營業利益成長率')
        self.股本 = data.get('financial_statement:股本')
        self.peg =self.pe/self.營業利益成長率
        self.tse_par = data.get('par_value_change_tse:twse_par_value_change_divide_ratio')
        self.otc_par = data.get('par_value_change_otc:otc_par_value_change_divide_ratio')


    def max_market_capitalization(self,quantile=0.6):
        #市值上限
        empty_df = pd.DataFrame(1, columns=self.close.columns, index=self.close.index)
        tse_divide_ratio = (empty_df*self.tse_par).fillna(1).cumprod()
        otc_divide_ratio = (empty_df*self.otc_par).fillna(1).cumprod()
        par_divide_ratio = tse_divide_ratio*otc_divide_ratio
        市值 = self.股本 * self.close / 10 * 1000 * par_divide_ratio
        condition = 市值 < 市值.quantile_row(quantile)
        return condition

    def min_market_capitalization(self,quantile=0.1):
        #市值下限
        empty_df = pd.DataFrame(1, columns=self.close.columns, index=self.close.index)
        tse_divide_ratio = (empty_df*self.tse_par).fillna(1).cumprod()
        otc_divide_ratio = (empty_df*self.otc_par).fillna(1).cumprod()
        par_divide_ratio = tse_divide_ratio*otc_divide_ratio
        市值 = self.股本 * self.close / 10 * 1000 * par_divide_ratio
        condition = 市值 > 市值.quantile_row(quantile)
        return condition


    def closing_price_below_market_classification(self,quantile=0.4):
        #低價股
         # 收盤價低於整體市場分級的40%
        condition = self.close <= self.close.quantile_row(quantile)
        return condition

    def closing_price_uper_market_classification(self,quantile=0.1):
        #高價股
        # 收盤價高於整體市場分級的10%
        condition = self.close >= self.close.quantile_row(quantile)
        return condition
    def all_time_high_revenue(self,rev_ma=2,rolling_window=12,rolling_period=6):
        rev_ma = self.revenue.average(rev_ma)
        condition = rev_ma == rev_ma.rolling(rolling_window, min_periods=rolling_period).max()
        return condition


    def ADLs_position(self,short_par=20,long_par=55):

        self.close_diff = self.close.diff()
        total_stocks = (~self.close.isna()).sum(1)
        rise_stocks = (self.close_diff>0).sum(1)
        ADLs = rise_stocks/total_stocks - 0.5
        short_ADLs_ma = ADLs.rolling(short_par).mean()
        long_ADLs_ma = ADLs.rolling(long_par).mean()
        cond = ~self.close.isna()
        cond1 = short_ADLs_ma >= long_ADLs_ma
        position = cond & cond1
        return position



    def small_inv(self,level = 5,datatype = '持有股數'):

        small_inv = FinlabDataFrame(self.inv[self.inv.持股分級.astype(int) <= level]
                                .reset_index().groupby(['date', 'stock_id'])
                                .agg({datatype: 'sum'})
                                .reset_index()
                                .pivot(index = 'date',columns= 'stock_id')[datatype])
        return small_inv

    def boss_inv(self,down_level = 9,  up_level= 15 ,datatype = '持有股數'):

        boss_inv =  FinlabDataFrame(self.inv[(self.inv.持股分級.astype(int) >= down_level) & (self.inv.持股分級.astype(int) <= up_level)]
                                        .reset_index()
                                        .groupby(['date', 'stock_id'])
                                        .agg({datatype: 'sum'})
                                        .reset_index()
                                        .pivot(index='date',columns= 'stock_id')[datatype])
        return boss_inv

    def compute_candle_volatility(self,timeperiod=20):


        bullish_candle = self.close >= self.open
        bullish_volatility = abs(self.close.shift() - self.open) + abs(self.open - self.low) + abs(self.low - self.high) + abs(self.high - self.close)
        bearish_volatility = abs(self.close.shift() - self.open) + abs(self.open - self.high) + abs(self.high-self.low) + abs(self.low - self.close)
        candle_volatility = FinlabDataFrame(np.nan, index=self.close.index, columns=self.close.columns)
        candle_volatility[bullish_candle] = bullish_volatility
        candle_volatility[~bullish_candle] = bearish_volatility
        volatility = candle_volatility.average(timeperiod) / self.close.average(timeperiod) * 100
        return volatility



In [5]:
from finlab import data
from finlab.dataframe import FinlabDataFrame

rev = data.get('monthly_revenue:當月營收')
close = data.get('price:收盤價')
vol = data.get('price:成交股數')
c= Condition()
h1 = c.small_inv(level = 5,datatype = '持有股數')
h2 = c.boss_inv(down_level = 9,  up_level= 15,datatype = '持有股數')
ratio = (h2 / (h1 + h2))
ratio2 = (h1 / (h1 + h2))


p1 = (FinlabDataFrame((ratio).diff(2)) *
    (close.notna()))

p2 = (FinlabDataFrame((ratio2).diff(2)) *
  (close.notna()))





s = (FinlabDataFrame(ratio.diff(6)).rank(axis=1, pct=True) *
    (close.notna()))





c1 = ((close / close.rolling(100).max())==1).sustain(4,1)

c3 = c.closing_price_uper_market_classification(quantile=0.1)
c4 = vol.rolling(5).mean() > vol.rolling(40).mean()
c6 = (s.rank(pct=True, axis=1) > 0.6)
c7 = p2 < 0



c8 = c.max_market_capitalization(quantile=0.7)
c9 = c.min_market_capitalization(quantile=0.1)

rev_ratio = (rev.rolling(2).mean() / rev.rolling(12).mean())

c10 = rev_ratio >1.05

small_inv_ = c.small_inv(level = 5,datatype = '占集保庫存數比例')

c11 = (
        (small_inv_==small_inv_.rolling(2).max()) # 賣出條件 八級以下散戶持續七周增加
        )

buy =  (
    c1 & #創新高
    c3 & #拒絕水餃
    c4 & #成交量
    c6 & #大戶不退場
    c7 & #散戶拋售
    c8 & #市值上限70％
    c9 & #市值下限10％
    c10   #營收成長
    )

small_inv_ = c.small_inv(level = 5,datatype = '占集保庫存數比例')

sell = (
        ((small_inv_==small_inv_.rolling(2).max()) ) # 賣出條件 八級以下散戶持續七周增加
        )
# sell = (
#         (close.average(5)<close.average(5).shift()).sustain(2) # 賣出條件 10ma連續兩日下彎
#         )
# sell = (
#         (close.rolling(2).min()<close.rolling(2).min().shift())#.sustain(2) # 賣出條件 八級以下散戶持續七周增加
#         )
position = buy.hold_until(exit=sell,
                          nstocks_limit=10,
                          rank=rev_ratio.replace([np.inf, -np.inf], np.nan).index_str_to_date(),stop_loss = 0.14
                          )



report = sim(resample = "W",
            position=position,
            position_limit = 0.13,
            trade_at_price='close',
            name='高速活塞',
            upload=True,
            mae_mfe_window=40,
            fee_ratio=1.425/1000/2)
report.display()
cc=report.get_trades()
cc.to_excel('crazylong.xlsx')

,annualized_rate_of_return,sharpe,max_drawdown,win_ratio
,72.49%,3.0,-23.11%,54.79%


,entry_date,exit_date,entry_sig_date,exit_sig_date,position,period,entry_index,exit_index,return,trade_price@entry_date,trade_price@exit_date,mae,gmfe,bmfe,mdd,pdays,weight,next_weights
stock_id,,,,,,,,,,,,,,,,,,
2374 佳能,2023-08-28,NaT,2023-08-25,NaT,0.111111,5.0,4028.0,-1.0,0.021459,23.30,23.8,-0.017167,0.021459,0.00000,-0.017167,1.0,0.111111,0.1
2488 漢平,2023-08-14,NaT,2023-08-11,2023-09-03,0.100000,15.0,4018.0,-1.0,0.100000,36.00,39.6,0.000000,0.100000,0.00000,-0.062338,14.0,0.111111,0.0
3043 科風,2023-08-21,NaT,2023-08-18,NaT,0.100000,10.0,4023.0,-1.0,0.101243,56.30,62.0,-0.023091,0.166963,0.00000,-0.056317,8.0,0.111111,0.1
3213 茂訊,2023-08-14,NaT,2023-08-11,NaT,0.100000,15.0,4018.0,-1.0,-0.014815,67.50,66.5,-0.080000,0.000000,0.00000,-0.080000,0.0,0.111111,0.1
4113 聯上,2023-07-03,NaT,2023-06-30,2023-09-03,0.100000,44.0,3989.0,-1.0,0.369942,17.30,23.7,-0.020231,0.433526,0.00289,-0.044355,31.0,0.111111,0.0
5215 科嘉-KY,2023-08-28,NaT,2023-08-25,2023-09-03,0.111111,5.0,4028.0,-1.0,0.130055,45.75,51.7,0.000000,0.130055,0.00000,0.000000,4.0,0.111111,0.0
5236 凌陽創新,2023-08-28,NaT,2023-08-25,NaT,0.111111,5.0,4028.0,-1.0,0.011450,131.00,132.5,0.000000,0.049618,0.00000,-0.036364,4.0,0.111111,0.1
6173 信昌電,2023-08-28,NaT,2023-08-25,NaT,0.111111,5.0,4028.0,-1.0,0.121844,45.55,51.1,0.000000,0.121844,0.00000,0.000000,4.0,0.111111,0.1
6538 倉和,2023-08-21,NaT,2023-08-18,2023-09-03,0.100000,10.0,4023.0,-1.0,0.047059,170.00,178.0,-0.047059,0.073529,0.00000,-0.047059,8.0,0.111111,0.0


,annualized_rate_of_return,sharpe,max_drawdown,win_ratio
,72.49%,3.0,-23.11%,54.79%


,entry_date,exit_date,entry_sig_date,exit_sig_date,position,period,entry_index,exit_index,return,trade_price@entry_date,trade_price@exit_date,mae,gmfe,bmfe,mdd,pdays,weight,next_weights
stock_id,,,,,,,,,,,,,,,,,,
2374 佳能,2023-08-28,NaT,2023-08-25,NaT,0.111111,5.0,4028.0,-1.0,0.021459,23.30,23.8,-0.017167,0.021459,0.00000,-0.017167,1.0,0.111111,0.1
2488 漢平,2023-08-14,NaT,2023-08-11,2023-09-03,0.100000,15.0,4018.0,-1.0,0.100000,36.00,39.6,0.000000,0.100000,0.00000,-0.062338,14.0,0.111111,0.0
3043 科風,2023-08-21,NaT,2023-08-18,NaT,0.100000,10.0,4023.0,-1.0,0.101243,56.30,62.0,-0.023091,0.166963,0.00000,-0.056317,8.0,0.111111,0.1
3213 茂訊,2023-08-14,NaT,2023-08-11,NaT,0.100000,15.0,4018.0,-1.0,-0.014815,67.50,66.5,-0.080000,0.000000,0.00000,-0.080000,0.0,0.111111,0.1
4113 聯上,2023-07-03,NaT,2023-06-30,2023-09-03,0.100000,44.0,3989.0,-1.0,0.369942,17.30,23.7,-0.020231,0.433526,0.00289,-0.044355,31.0,0.111111,0.0
5215 科嘉-KY,2023-08-28,NaT,2023-08-25,2023-09-03,0.111111,5.0,4028.0,-1.0,0.130055,45.75,51.7,0.000000,0.130055,0.00000,0.000000,4.0,0.111111,0.0
5236 凌陽創新,2023-08-28,NaT,2023-08-25,NaT,0.111111,5.0,4028.0,-1.0,0.011450,131.00,132.5,0.000000,0.049618,0.00000,-0.036364,4.0,0.111111,0.1
6173 信昌電,2023-08-28,NaT,2023-08-25,NaT,0.111111,5.0,4028.0,-1.0,0.121844,45.55,51.1,0.000000,0.121844,0.00000,0.000000,4.0,0.111111,0.1
6538 倉和,2023-08-21,NaT,2023-08-18,2023-09-03,0.100000,10.0,4023.0,-1.0,0.047059,170.00,178.0,-0.047059,0.073529,0.00000,-0.047059,8.0,0.111111,0.0


In [6]:
# (small_inv_==small_inv_.rolling(2).max()) | ~c6